### YOLOv8
A computer vision model architecture for detection, classification, segmentation, and more.

In [1]:
from inference import InferencePipeline
from inference.core.interfaces.stream.sinks import render_boxes

ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.To suppress this warning, set QWEN_2_5_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM2 model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM2_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support CLIP model. Use pip install 'inference[clip]' to install missing requirements.To suppress this warning, set CORE_MODEL_CLIP_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support Gaze Detection model. Use pip install 'inference[gaze]' to install mi

In [28]:
# моя любіма

import cv2
import numpy as np
import pyautogui
from ultralytics import YOLO
import math
from deep_sort_realtime.deepsort_tracker import DeepSort

# model
model = YOLO("yolov8s.pt") # n, s, m, l, x 

# object classes
classNames = ["person", "cat", "dog"]
# if we need  "cup", "bottle", "laptop", "chair", "diningtable", "cell phone", "potted plant"

# DeepSORT трекер
tracker = DeepSort(max_age=30)

frame_idx = 0
process_every_n = 3   # обробляти кожен 3-й кадр
last_results = None   # сюди зберігаємо останні результати

# video coffee_shop, coffee, cats
video_path = "coffee_shop.mp4"

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# налаштування запису відео 
frame_width, frame_height = pyautogui.size()  
out = cv2.VideoWriter(
    'screen_detection.mp4',                 
    cv2.VideoWriter_fourcc(*'mp4v'),        
    15,                                     
    (frame_width, frame_height)             
)

while True:
    use_screen = True  # True -> знімок екрану, False -> відео

    if use_screen:
        window_name = "Screen Detection"
    else:
        window_name = "Video Detection"

    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 960, 540)

    if use_screen:
        screenshot = pyautogui.screenshot()
        frame = np.array(screenshot)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    else:
        ret, frame = cap.read()
        if not ret:
            break  # відео закінчилося

    frame_idx += 1

    # оновлюємо результати тільки кожен n-й кадр
    if frame_idx % process_every_n == 0:
        results = model(frame, imgsz=640, stream=False)
        last_results = results

    # малюємо бокси тільки для потрібних класів
    if last_results is not None:
        for r in last_results:
            boxes = r.boxes
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id] # отримуємо ім'я класу
                if cls_name in classNames:     # перевіряємо, чи потрібний клас
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    confidence = math.ceil((box.conf[0]*100))/100
                    
                    # малюємо прямокутник
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
                    # підпис з назвою класу і confidence
                    cv2.putText(frame, f"{cls_name} {confidence}", (x1, y1-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

    cv2.imshow("Screen Detection", frame)
    out.write(frame)  # записуємо кадр у файл

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# завершення запису
out.release()
cv2.destroyAllWindows()



In [ ]:
# ця версія не створює багато рамок але є шум 

import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO

# ---------------------------
# Параметри
# ---------------------------
video_path = "coffee_shop.mp4"
model = YOLO("yolov8n.pt")
classNames = ["person"]
min_conf = 0.5          # мінімальна confidence для боксу
min_box_size = 20       # мінімальний розмір боксу для людини
track_history = defaultdict(list)  # зберігаємо координати центрів

# ---------------------------
# Відкриваємо відео
# ---------------------------
cap = cv2.VideoCapture(video_path)

cv2.namedWindow("YOLOv8 Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Tracking", 960, 540)

frame_idx = 0
process_every_n = 2  # обробляти кожен 2-й кадр

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1

    if frame_idx % process_every_n == 0:
        # детекція і трекінг людей
        results = model.track(frame, persist=True, imgsz=640)

        # обробка тільки першого результату
        r = results[0]
        boxes = r.boxes
        if boxes is not None:
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id]
                conf = float(box.conf[0])
                if cls_name in classNames and conf >= min_conf:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    if w >= min_box_size and h >= min_box_size:
                        track_id = int(box.id[0])

                        # малюємо рамку
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
                        cv2.putText(frame, f"ID {track_id}", (x1, y1-15),
                                    cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 255, 0), 5)

                        # зберігаємо центр боксу для відстеження
                        cx, cy = (x1+x2)//2, (y1+y2)//2
                        track_history[track_id].append((cx, cy))
                        if len(track_history[track_id]) > 2:
                            points = np.array(track_history[track_id][-15:], np.int32).reshape((-1,1,2))  # тільки останні 15
                            cv2.polylines(frame, [points], False, (200,200,200), 2)


                        # малюємо шлях треку
                        points = np.array(track_history[track_id], np.int32).reshape((-1,1,2))
                        cv2.polylines(frame, [points], False, (200,200,200), 2)

    cv2.imshow("YOLOv8 Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
# трекаєм час людей за айді

import cv2
import numpy as np
import time
import csv
from collections import defaultdict
from ultralytics import YOLO
from ultralytics.utils import LOGGER

# Вимикаємо внутрішні логи Ultralytics (щоб не було зайвого виводу)
LOGGER.setLevel(40)  # ERROR і вище

# ---------------------------
# Параметри
# ---------------------------
video_path = "coffee_shop.mp4"
model = YOLO("yolov8n.pt")
classNames = ["person", "dog", "cat"]
min_conf = 0.3          # мінімальна confidence для боксу
min_box_size = 20       # мінімальний розмір боксу для людини
track_history = defaultdict(list)  # зберігаємо координати центрів

# Лог часу
entry_times = {}   # track_id -> час появи
last_seen = {}     # track_id -> останній кадр
timeout = 5        # скільки секунд чекати, щоб вважати що людина пішла

# CSV лог
log_file = "visitors_log.csv"
with open(log_file, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["track_id", "start_time", "end_time", "duration_sec"])

# ---------------------------
# Відкриваємо відео
# ---------------------------
cap = cv2.VideoCapture(video_path)

cv2.namedWindow("YOLOv8 Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Tracking", 960, 540)

frame_idx = 0
process_every_n = 1  # обробляти кожен кадр

while True:
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    frame_idx += 1
    now = time.time()

    if frame_idx % process_every_n == 0:
        # детекція і трекінг людей
        results = model.track(frame, persist=True, imgsz=640)
        r = results[0]
        boxes = r.boxes

        seen_ids = set()

        detected_counts = {name: 0 for name in classNames}
        
        if boxes is not None:
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id]
                conf = float(box.conf[0])
                if cls_name in classNames and conf >= min_conf:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    detected_counts[cls_name] += 1
                    if w >= min_box_size and h >= min_box_size:
                        track_id = int(box.id[0])
                        seen_ids.add(track_id)

                        # Перша поява
                        if track_id not in entry_times:
                            entry_times[track_id] = now
                        
                        last_seen[track_id] = now

                        # малюємо рамку
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                        cv2.putText(frame, f"{cls_name} ID {track_id}", (x1, y1-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)


                        # зберігаємо центр боксу
                        cx, cy = (x1+x2)//2, (y1+y2)//2
                        track_history[track_id].append((cx, cy))
                        if len(track_history[track_id]) > 15:
                            track_history[track_id].pop(0)

                        # малюємо шлях
                        points = np.array(track_history[track_id], np.int32).reshape((-1,1,2))
                        cv2.polylines(frame, [points], False, (200,200,200), 2)

        # перевіряємо, хто зник
        to_remove = []
        for track_id, last_time in last_seen.items():
            if now - last_time > timeout:
                start_time = entry_times[track_id]
                end_time = last_time
                duration = end_time - start_time

                # запис у CSV
                with open(log_file, mode="a", newline="", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow([
                        track_id,
                        time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)),
                        time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time)),
                        round(duration, 1)
                    ])

                to_remove.append(track_id)

        for track_id in to_remove:
            del entry_times[track_id]
            del last_seen[track_id]

        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        fps = 1000 / elapsed_ms if elapsed_ms > 0 else 0

        # Формуємо повідомлення про знайдені об'єкти
        log_message = ", ".join([f"{count} {name}s" for name, count in detected_counts.items() if count > 0])
        if log_message:
            print(f"Detected: {log_message} | Inference time: {elapsed_ms:.1f} ms | FPS: {fps:.1f}")
        else:
            print(f"No targets detected | Inference time: {elapsed_ms:.1f} ms | FPS: {fps:.1f}")

    cv2.imshow("YOLOv8 Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Detected: 10 persons | Inference time: 216.7 ms | FPS: 4.6
Detected: 9 persons | Inference time: 175.6 ms | FPS: 5.7
Detected: 9 persons | Inference time: 164.5 ms | FPS: 6.1
Detected: 9 persons | Inference time: 191.8 ms | FPS: 5.2
Detected: 9 persons | Inference time: 166.3 ms | FPS: 6.0
Detected: 10 persons | Inference time: 160.6 ms | FPS: 6.2
Detected: 9 persons | Inference time: 162.5 ms | FPS: 6.2
Detected: 9 persons | Inference time: 164.3 ms | FPS: 6.1
Detected: 9 persons | Inference time: 168.9 ms | FPS: 5.9
Detected: 9 persons | Inference time: 166.0 ms | FPS: 6.0
Detected: 9 persons | Inference time: 163.9 ms | FPS: 6.1
Detected: 9 persons | Inference time: 164.1 ms | FPS: 6.1
Detected: 9 persons | Inference time: 163.1 ms | FPS: 6.1
Detected: 10 persons | Inference time: 164.6 ms | FPS: 6.1
Detected: 10 persons | Inference time: 165.4 ms | FPS: 6.0
Detected: 10 persons | Inference time: 164.6 ms | FPS: 6.1
Detected: 10 persons | Inference time: 182.9 ms | FPS: 5.5
Detected

In [19]:
# кольорове із записом і логами 

import cv2
import numpy as np
import time
import csv
import random
from collections import defaultdict
from ultralytics import YOLO
from ultralytics.utils import LOGGER

# Вимикаємо внутрішні логи Ultralytics
LOGGER.setLevel(40)

# ---------------------------
# Параметри
# ---------------------------
video_path = "coffee_shop.mp4"
model = YOLO("yolov8n.pt")
classNames = ["person", "dog", "cat"]
min_conf = 0.5
min_box_size = 20
timeout = 5  # секунд очікування, щоб вважати що об'єкт зник

# ---------------------------
# Лог і статистика
# ---------------------------
entry_times = {}      # track_id -> час появи
last_seen = {}        # track_id -> останній раз бачили
durations = {}        # track_id -> загальний час
track_history = defaultdict(list)  # історія координат

# Унікальні кольори для ID
id_colors = {}

def get_color(track_id):
    if track_id not in id_colors:
        id_colors[track_id] = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255)
        )
    return id_colors[track_id]

# CSV лог
log_file = "visitors_log.csv"
f = open(log_file, mode="w", newline="", encoding="utf-8")
writer = csv.writer(f)
writer.writerow(["track_id", "start_time", "end_time", "duration_sec"])

# ---------------------------
# Відкриваємо відео
# ---------------------------
cap = cv2.VideoCapture(video_path)

cv2.namedWindow("YOLOv8 Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Tracking", 960, 540)

frame_idx = 0
process_every_n = 1

start_global = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    now = time.perf_counter()

    if frame_idx % process_every_n == 0:
        start_infer = time.perf_counter()

        results = model.track(frame, persist=True, imgsz=640)
        r = results[0]
        boxes = r.boxes

        detected_counts = {name: 0 for name in classNames}
        seen_ids = set()

        if boxes is not None:
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id]
                conf = float(box.conf[0])
                if cls_name in classNames and conf >= min_conf:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    detected_counts[cls_name] += 1

                    if w >= min_box_size and h >= min_box_size and box.id is not None:
                        track_id = int(box.id[0])
                        seen_ids.add(track_id)

                        # Реєстрація появи
                        if track_id not in entry_times:
                            entry_times[track_id] = time.time()

                        last_seen[track_id] = time.time()

                        # Малюємо рамку
                        color = get_color(track_id)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                        cv2.putText(frame, f"{cls_name} ID {track_id}",
                                    (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX,
                                    0.8, color, 2)

                        # Малюємо шлях
                        cx, cy = (x1+x2)//2, (y1+y2)//2
                        track_history[track_id].append((cx, cy))
                        if len(track_history[track_id]) > 15:
                            track_history[track_id].pop(0)
                        points = np.array(track_history[track_id], np.int32).reshape((-1,1,2))
                        cv2.polylines(frame, [points], False, color, 2)

        # Перевіряємо, хто зник
        to_remove = []
        for track_id, last_time_seen in list(last_seen.items()):
            if time.time() - last_time_seen > timeout:
                start_time = entry_times[track_id]
                end_time = last_time_seen
                duration = end_time - start_time

                durations[track_id] = duration

                writer.writerow([
                    track_id,
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)),
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time)),
                    round(duration, 1)
                ])

                to_remove.append(track_id)

        for tid in to_remove:
            entry_times.pop(tid, None)
            last_seen.pop(tid, None)

        # FPS
        end_infer = time.perf_counter()
        elapsed_ms = (end_infer - start_infer) * 1000
        fps = 1000 / elapsed_ms if elapsed_ms > 0 else 0

        log_message = ", ".join([f"{count} {name}(s)" for name, count in detected_counts.items() if count > 0])
        if log_message:
            print(f"Detected: {log_message} | {elapsed_ms:.1f} ms | FPS: {fps:.1f}")
        else:
            print(f"No targets detected | {elapsed_ms:.1f} ms | FPS: {fps:.1f}")

    cv2.imshow("YOLOv8 Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
f.close()
cv2.destroyAllWindows()

# ---------------------------
# Зведена статистика
# ---------------------------
if durations:
    avg_time = sum(durations.values()) / len(durations)
    print(f"\nЗагалом відвідувачів: {len(durations)}")
    print(f"Середній час перебування: {avg_time:.1f} секунд")
else:
    print("\nВідвідувачів не зафіксовано.")


Detected: 8 person(s) | 316.4 ms | FPS: 3.2
Detected: 8 person(s) | 232.2 ms | FPS: 4.3
Detected: 8 person(s) | 212.9 ms | FPS: 4.7
Detected: 8 person(s) | 206.9 ms | FPS: 4.8
Detected: 8 person(s) | 223.6 ms | FPS: 4.5
Detected: 8 person(s) | 222.5 ms | FPS: 4.5
Detected: 8 person(s) | 234.2 ms | FPS: 4.3
Detected: 8 person(s) | 303.2 ms | FPS: 3.3
Detected: 8 person(s) | 223.4 ms | FPS: 4.5
Detected: 8 person(s) | 224.0 ms | FPS: 4.5
Detected: 8 person(s) | 209.8 ms | FPS: 4.8
Detected: 8 person(s) | 222.3 ms | FPS: 4.5
Detected: 8 person(s) | 209.7 ms | FPS: 4.8
Detected: 8 person(s) | 223.3 ms | FPS: 4.5
Detected: 8 person(s) | 230.0 ms | FPS: 4.3
Detected: 8 person(s) | 211.6 ms | FPS: 4.7
Detected: 8 person(s) | 249.6 ms | FPS: 4.0
Detected: 8 person(s) | 257.0 ms | FPS: 3.9
Detected: 8 person(s) | 248.6 ms | FPS: 4.0
Detected: 8 person(s) | 212.5 ms | FPS: 4.7
Detected: 8 person(s) | 228.7 ms | FPS: 4.4
Detected: 8 person(s) | 209.8 ms | FPS: 4.8
Detected: 7 person(s) | 243.6 ms

In [ ]:
# люди добре зберігаються але багато рамок 

import cv2
import numpy as np
import os
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import hashlib

# ---------------------------
# Параметри
# ---------------------------
video_path = "coffee_shop.mp4"
model = YOLO("yolov8n.pt")
classNames = ["person"]
tracker = DeepSort(max_age=30)

frame_idx = 0
process_every_n = 3

# Папка для збереження фото
save_dir = "detections"
os.makedirs(save_dir, exist_ok=True)

# Для відстеження попереднього стану людини
last_state = {}  # {track_id: (x, y, w, h)}
max_photos_per_person = 10

# ---------------------------
# Відкриваємо відео
# ---------------------------
cap = cv2.VideoCapture(video_path)

cv2.namedWindow("Video Detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Video Detection", 960, 540)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1

    # оновлюємо тільки кожен n-й кадр
    if frame_idx % process_every_n == 0:
        results = model(frame, imgsz=640, stream=False)

    if results is not None:
        detections = []
        for r in results:
            for box in r.boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id]
                if cls_name in classNames:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls_name))

        # запускаємо DeepSORT
        tracks = tracker.update_tracks(detections, frame=frame)

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            w, h = x2 - x1, y2 - y1

            # Малюємо прямокутник + ID
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
            cv2.putText(frame, f"ID {track_id}", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

            # --- збереження фото тільки при зміні стану ---
            person_crop = frame[y1:y2, x1:x2]
            if person_crop.size == 0:
                continue

            person_dir = os.path.join(save_dir, f"ID{track_id}")
            os.makedirs(person_dir, exist_ok=True)

            # хеш кадру, щоб уникнути дублів
            img_bytes = cv2.imencode('.jpg', person_crop)[1].tobytes()
            img_hash = hashlib.md5(img_bytes).hexdigest()

            # перевірка на попередній стан
            prev = last_state.get(track_id)
            state_changed = True
            if prev:
                px, py, pw, ph = prev
                # якщо зміна менша ніж 10% по ширині/висоті та положенню — не зберігаємо
                if abs(px - x1) < 0.1*pw and abs(py - y1) < 0.1*ph and abs(pw - w) < 0.1*pw and abs(ph - h) < 0.1*ph:
                    state_changed = False

            # обмеження на 10 фото
            existing_photos = len(os.listdir(person_dir))
            if state_changed and existing_photos < max_photos_per_person:
                save_path = os.path.join(person_dir, f"{existing_photos+1}_{img_hash}.jpg")
                cv2.imwrite(save_path, person_crop)
                last_state[track_id] = (x1, y1, w, h)

    cv2.imshow("Video Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [18]:
import cv2
import numpy as np
import os
import hashlib
from collections import defaultdict
from ultralytics import YOLO

# ---------------------------
# Параметри
# ---------------------------
video_path = "C:/Users/Angelika/Videos/Captures/coffee.mp4"
model = YOLO("yolov8n.pt")
classNames = ["person"] 
min_conf = 0.5
min_box_size = 20
track_history = defaultdict(list)

# Збереження фото
save_dir = "detections"
os.makedirs(save_dir, exist_ok=True)
last_state = {}  # {track_id: (x, y, w, h)}
max_photos_per_person = 10

# ---------------------------
# Відкриваємо відео
# ---------------------------
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)  # FPS відео
cv2.namedWindow("YOLOv8 Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Tracking", 960, 540)

frame_idx = 0
process_every_n = 1

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    video_time = frame_idx / fps  # час у секундах

    if frame_idx % process_every_n == 0:
        results = model.track(frame, persist=True, imgsz=640)
        r = results[0]
        boxes = r.boxes

        if boxes is not None:
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id]
                conf = float(box.conf[0])

                if cls_name in classNames and conf >= min_conf:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    if w >= min_box_size and h >= min_box_size:
                        track_id = int(box.id[0])

                        # Малюємо рамку
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                        cv2.putText(frame, f"ID {track_id}", (x1, y1-10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

                        # Центр та траєкторія
                        cx, cy = (x1+x2)//2, (y1+y2)//2
                        track_history[track_id].append((cx, cy))
                        if len(track_history[track_id]) > 2:
                            points = np.array(track_history[track_id][-15:], np.int32).reshape((-1,1,2))
                            cv2.polylines(frame, [points], False, (200,200,200), 2)

                        # ---- Збереження фото ----
                        person_crop = frame[y1:y2, x1:x2]
                        if person_crop.size == 0:
                            continue

                        person_dir = os.path.join(save_dir, f"ID{track_id}")
                        os.makedirs(person_dir, exist_ok=True)

                        # Хеш кадру
                        img_bytes = cv2.imencode('.jpg', person_crop)[1].tobytes()
                        img_hash = hashlib.md5(img_bytes).hexdigest()

                        # Перевірка на зміни
                        prev = last_state.get(track_id)
                        state_changed = True
                        if prev:
                            px, py, pw, ph = prev
                            if (abs(px - x1) < 0.1*pw and abs(py - y1) < 0.1*ph and 
                                abs(pw - w) < 0.1*pw and abs(ph - h) < 0.1*ph):
                                state_changed = False

                        existing_photos = len(os.listdir(person_dir))
                        if state_changed and existing_photos < max_photos_per_person:
                            save_path = os.path.join(person_dir, f"{int(video_time)}s_{img_hash}.jpg")
                            cv2.imwrite(save_path, person_crop)
                            last_state[track_id] = (x1, y1, w, h)

    cv2.imshow("YOLOv8 Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [23]:
import cv2
import numpy as np
import time
import csv
import random
import os
import hashlib
from collections import defaultdict
from ultralytics import YOLO
from ultralytics.utils import LOGGER

# Вимикаємо внутрішні логи Ultralytics
LOGGER.setLevel(40)

# ---------------------------
# Параметри
# ---------------------------
video_path = "coffee_shop.mp4"
model = YOLO("yolov8n.pt")
classNames = ["person", "dog", "cat"]
min_conf = 0.5
min_box_size = 20
timeout = 5  # секунд очікування

# Збереження фото
save_dir = "detections"
os.makedirs(save_dir, exist_ok=True)
max_photos_per_person = 10
last_state = {}  # {track_id: (cx, cy)}

# ---------------------------
# Лог і статистика
# ---------------------------
entry_times = {}
last_seen = {}
durations = {}
track_history = defaultdict(list)
id_colors = {}

def get_color(track_id):
    if track_id not in id_colors:
        id_colors[track_id] = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255)
        )
    return id_colors[track_id]

# CSV лог
log_file = "visitors_log.csv"
f = open(log_file, mode="w", newline="", encoding="utf-8")
writer = csv.writer(f)
writer.writerow(["track_id", "start_time", "end_time", "duration_sec"])

# ---------------------------
# Відкриваємо відео
# ---------------------------
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

cv2.namedWindow("YOLOv8 Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Tracking", 960, 540)

frame_idx = 0
process_every_n = 1

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    video_time = frame_idx / fps
    now = time.perf_counter()

    if frame_idx % process_every_n == 0:
        start_infer = time.perf_counter()
        results = model.track(frame, persist=True, imgsz=640)
        r = results[0]
        boxes = r.boxes

        detected_counts = {name: 0 for name in classNames}
        seen_ids = set()

        if boxes is not None:
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id]
                conf = float(box.conf[0])
                if cls_name in classNames and conf >= min_conf:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    detected_counts[cls_name] += 1

                    if w >= min_box_size and h >= min_box_size and box.id is not None:
                        track_id = int(box.id[0])
                        seen_ids.add(track_id)

                        # Реєстрація появи
                        if track_id not in entry_times:
                            entry_times[track_id] = time.time()

                        last_seen[track_id] = time.time()

                        # Малюємо рамку
                        color = get_color(track_id)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                        cv2.putText(frame, f"{cls_name} ID {track_id}",
                                    (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX,
                                    0.8, color, 2)

                        # Малюємо шлях
                        cx, cy = (x1+x2)//2, (y1+y2)//2
                        track_history[track_id].append((cx, cy))
                        if len(track_history[track_id]) > 15:
                            track_history[track_id].pop(0)
                        points = np.array(track_history[track_id], np.int32).reshape((-1,1,2))
                        cv2.polylines(frame, [points], False, color, 2)

                        # --- Збереження фото при русі ---
                        person_crop = frame[y1:y2, x1:x2]
                        if person_crop.size > 0:
                            person_dir = os.path.join(save_dir, f"ID{track_id}")
                            os.makedirs(person_dir, exist_ok=True)

                            prev = last_state.get(track_id)
                            moved = True
                            if prev:
                                px, py = prev
                                if abs(cx - px) < 5 and abs(cy - py) < 5:  # майже не рухався
                                    moved = False

                            existing_photos = len(os.listdir(person_dir))
                            if moved and existing_photos < max_photos_per_person:
                                img_bytes = cv2.imencode('.jpg', person_crop)[1].tobytes()
                                img_hash = hashlib.md5(img_bytes).hexdigest()
                                save_path = os.path.join(person_dir, f"{frame_idx}_{int(video_time)}s_{img_hash}.jpg")
                                cv2.imwrite(save_path, person_crop)
                                last_state[track_id] = (cx, cy)

        # Перевіряємо, хто зник
        to_remove = []
        for track_id, last_time_seen in list(last_seen.items()):
            if time.time() - last_time_seen > timeout:
                start_time = entry_times[track_id]
                end_time = last_time_seen
                duration = end_time - start_time
                durations[track_id] = duration
                writer.writerow([
                    track_id,
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)),
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time)),
                    round(duration, 1)
                ])
                to_remove.append(track_id)

        for tid in to_remove:
            entry_times.pop(tid, None)
            last_seen.pop(tid, None)

        # FPS
        end_infer = time.perf_counter()
        elapsed_ms = (end_infer - start_infer) * 1000
        fps_val = 1000 / elapsed_ms if elapsed_ms > 0 else 0

        log_message = ", ".join([f"{count} {name}(s)" for name, count in detected_counts.items() if count > 0])
        if log_message:
            print(f"Detected: {log_message} | {elapsed_ms:.1f} ms | FPS: {fps_val:.1f}")
        else:
            print(f"No targets detected | {elapsed_ms:.1f} ms | FPS: {fps_val:.1f}")

    cv2.imshow("YOLOv8 Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
f.close()
cv2.destroyAllWindows()

# ---------------------------
# Зведена статистика
# ---------------------------
if durations:
    avg_time = sum(durations.values()) / len(durations)
    print(f"\nЗагалом відвідувачів: {len(durations)}")
    print(f"Середній час перебування: {avg_time:.1f} секунд")
else:
    print("\nВідвідувачів не зафіксовано.")


Detected: 8 person(s) | 247.7 ms | FPS: 4.0
Detected: 8 person(s) | 175.9 ms | FPS: 5.7
Detected: 8 person(s) | 166.7 ms | FPS: 6.0
Detected: 8 person(s) | 168.1 ms | FPS: 5.9
Detected: 8 person(s) | 249.7 ms | FPS: 4.0
Detected: 8 person(s) | 167.8 ms | FPS: 6.0
Detected: 8 person(s) | 168.9 ms | FPS: 5.9
Detected: 8 person(s) | 177.9 ms | FPS: 5.6
Detected: 8 person(s) | 171.5 ms | FPS: 5.8
Detected: 8 person(s) | 168.0 ms | FPS: 6.0
Detected: 8 person(s) | 162.7 ms | FPS: 6.1
Detected: 8 person(s) | 170.8 ms | FPS: 5.9
Detected: 8 person(s) | 169.7 ms | FPS: 5.9
Detected: 8 person(s) | 168.9 ms | FPS: 5.9
Detected: 8 person(s) | 172.7 ms | FPS: 5.8
Detected: 8 person(s) | 168.1 ms | FPS: 6.0
Detected: 8 person(s) | 171.0 ms | FPS: 5.8
Detected: 8 person(s) | 168.0 ms | FPS: 6.0
Detected: 8 person(s) | 168.8 ms | FPS: 5.9
Detected: 8 person(s) | 172.5 ms | FPS: 5.8
Detected: 8 person(s) | 167.4 ms | FPS: 6.0
Detected: 8 person(s) | 165.0 ms | FPS: 6.1
Detected: 7 person(s) | 169.3 ms

In [ ]:
# запис в файл поганий
import cv2
import numpy as np
import time
import csv
import random
import os
import hashlib
from collections import defaultdict
from ultralytics import YOLO
from ultralytics.utils import LOGGER

# Вимикаємо внутрішні логи Ultralytics
LOGGER.setLevel(40)

# ---------------------------
# Параметри
# ---------------------------
video_path = "coffee_shop.mp4"
model = YOLO("yolov8n.pt")
classNames = ["person", "dog", "cat"]
min_conf = 0.5
min_box_size = 20
timeout = 5  # секунд очікування

# Збереження фото
save_dir = "detections"
os.makedirs(save_dir, exist_ok=True)
max_photos_per_person = 10
last_state = {}  # {track_id: (cx, cy)}

# ---------------------------
# Лог і статистика
# ---------------------------
entry_times = {}
last_seen = {}
durations = {}
track_history = defaultdict(list)
id_colors = {}

def get_color(track_id):
    if track_id not in id_colors:
        id_colors[track_id] = (
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255)
        )
    return id_colors[track_id]

# CSV лог
log_file = "visitors_log.csv"
f = open(log_file, mode="w", newline="", encoding="utf-8")
writer = csv.writer(f)
writer.writerow(["track_id", "start_time", "end_time", "duration_sec"])

# ---------------------------
# Відкриваємо відео
# ---------------------------
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

cv2.namedWindow("YOLOv8 Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("YOLOv8 Tracking", 960, 540)

frame_idx = 0
process_every_n = 1

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    video_time = frame_idx / fps
    now = time.perf_counter()

    if frame_idx % process_every_n == 0:
        start_infer = time.perf_counter()
        results = model.track(frame, persist=True, imgsz=640)
        r = results[0]
        boxes = r.boxes

        detected_counts = {name: 0 for name in classNames}
        seen_ids = set()

        if boxes is not None:
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = model.names[cls_id]
                conf = float(box.conf[0])
                if cls_name in classNames and conf >= min_conf:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    detected_counts[cls_name] += 1

                    if w >= min_box_size and h >= min_box_size and box.id is not None:
                        track_id = int(box.id[0])
                        seen_ids.add(track_id)

                        # Реєстрація появи
                        if track_id not in entry_times:
                            entry_times[track_id] = time.time()

                        last_seen[track_id] = time.time()

                        # Малюємо рамку
                        color = get_color(track_id)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                        cv2.putText(frame, f"{cls_name} ID {track_id}",
                                    (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX,
                                    0.8, color, 2)

                        # Малюємо шлях
                        cx, cy = (x1+x2)//2, (y1+y2)//2
                        track_history[track_id].append((cx, cy))
                        if len(track_history[track_id]) > 15:
                            track_history[track_id].pop(0)
                        points = np.array(track_history[track_id], np.int32).reshape((-1,1,2))
                        cv2.polylines(frame, [points], False, color, 2)

                        # --- Збереження фото при русі ---
                        person_crop = frame[y1:y2, x1:x2]
                        if person_crop.size > 0:
                            person_dir = os.path.join(save_dir, f"ID{track_id}")
                            os.makedirs(person_dir, exist_ok=True)

                            prev = last_state.get(track_id)
                            moved = True
                            if prev:
                                px, py = prev
                                if abs(cx - px) < 5 and abs(cy - py) < 5:  # майже не рухався
                                    moved = False

                            existing_photos = len(os.listdir(person_dir))
                            if moved and existing_photos < max_photos_per_person:
                                img_bytes = cv2.imencode('.jpg', person_crop)[1].tobytes()
                                img_hash = hashlib.md5(img_bytes).hexdigest()
                                save_path = os.path.join(person_dir, f"{frame_idx}_{int(video_time)}s_{img_hash}.jpg")
                                cv2.imwrite(save_path, person_crop)
                                last_state[track_id] = (cx, cy)

        # Перевіряємо, хто зник
        to_remove = []
        for track_id, last_time_seen in list(last_seen.items()):
            if time.time() - last_time_seen > timeout:
                start_time = entry_times[track_id]
                end_time = last_time_seen
                duration = end_time - start_time
                durations[track_id] = duration
                writer.writerow([
                    track_id,
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(start_time)),
                    time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time)),
                    round(duration, 1)
                ])
                to_remove.append(track_id)

        for tid in to_remove:
            entry_times.pop(tid, None)
            last_seen.pop(tid, None)

        # FPS
        end_infer = time.perf_counter()
        elapsed_ms = (end_infer - start_infer) * 1000
        fps_val = 1000 / elapsed_ms if elapsed_ms > 0 else 0

        log_message = ", ".join([f"{count} {name}(s)" for name, count in detected_counts.items() if count > 0])
        if log_message:
            print(f"Detected: {log_message} | {elapsed_ms:.1f} ms | FPS: {fps_val:.1f}")
        else:
            print(f"No targets detected | {elapsed_ms:.1f} ms | FPS: {fps_val:.1f}")

    cv2.imshow("YOLOv8 Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
f.close()
cv2.destroyAllWindows()


Detected: 8 person(s) | 257.5 ms | FPS: 3.9
Detected: 8 person(s) | 186.2 ms | FPS: 5.4
Detected: 8 person(s) | 168.1 ms | FPS: 5.9
Detected: 8 person(s) | 167.8 ms | FPS: 6.0
Detected: 8 person(s) | 169.6 ms | FPS: 5.9
Detected: 8 person(s) | 166.7 ms | FPS: 6.0
Detected: 8 person(s) | 166.9 ms | FPS: 6.0
Detected: 8 person(s) | 165.0 ms | FPS: 6.1
Detected: 8 person(s) | 165.1 ms | FPS: 6.1
Detected: 8 person(s) | 166.0 ms | FPS: 6.0
Detected: 8 person(s) | 164.8 ms | FPS: 6.1
Detected: 8 person(s) | 171.3 ms | FPS: 5.8
Detected: 8 person(s) | 166.0 ms | FPS: 6.0
Detected: 8 person(s) | 164.9 ms | FPS: 6.1
Detected: 8 person(s) | 177.8 ms | FPS: 5.6
Detected: 8 person(s) | 167.5 ms | FPS: 6.0
Detected: 8 person(s) | 167.4 ms | FPS: 6.0
Detected: 8 person(s) | 172.1 ms | FPS: 5.8
Detected: 8 person(s) | 167.0 ms | FPS: 6.0
Detected: 8 person(s) | 169.2 ms | FPS: 5.9
Detected: 8 person(s) | 165.1 ms | FPS: 6.1
Detected: 8 person(s) | 169.2 ms | FPS: 5.9
Detected: 7 person(s) | 162.9 ms